# Environment Setup for Soccer Tracking Pipeline

This notebook helps you set up the environment for running the soccer tracking pipeline on Google Colab with remote VM access via VS Code SSH.

## 1. Install Dependencies

In [1]:
# Install PyTorch with CUDA support (automatically detects available CUDA version)
!pip install torch torchvision --extra-index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 96.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    U

In [2]:
# Install computer vision and tracking libraries
!pip install ultralytics boxmot opencv-python numpy scipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 70.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.6/61.6 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 104.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.4/276.4 kB 23.8 MB/s eta 0:00:00
  Created wheel for filterpy: filename=filterpy-1.4.5-py3-none-any.whl size=110460 sha256=e333f86e5f8c9d3b057deb8a0fcbfa6318a50f6773336a146229c8b38736b6f2
  Stored in directory: /root/.cache/pip/wheels/12/dc/3c/e12983eac132d00f82a20c6cbe7b42ce6e96190ef8

In [3]:
# Install evaluation library
!pip install -q git+https://github.com/JonathonLuiten/TrackEval.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [4]:
# Install additional utilities
!pip install tqdm matplotlib pandas pyyaml

## 2. Mount Google Drive (for data access)

In [5]:
# Mount Google Drive for dataset access
try:
    from google.colab import drive
    drive.mount('/content/drive')
    print("Google Drive mounted successfully!")

    # Check if soccer data exists
    import os
    gdrive_path = "/content/drive/MyDrive/SOCCER_DATA"
    if os.path.exists(gdrive_path):
        print(f"Soccer data found at: {gdrive_path}")
        print("Available datasets:")
        for item in os.listdir(gdrive_path):
            print(f"  - {item}")
    else:
        print(f"No soccer data found at: {gdrive_path}")
        print("Please upload your dataset to Google Drive first.")

except ImportError:
    print("Not running in Google Colab. Skipping drive mount.")
except Exception as e:
    print(f"Error mounting drive: {e}")

Mounted at /content/drive
Google Drive mounted successfully!
Soccer data found at: /content/drive/MyDrive/SOCCER_DATA
Available datasets:
  - deepsort_dataset_train
  - deepsort_dataset_test


## 3. Set Up the Project

In [13]:
# Clone your repository if it's not already present
import os

# Define the project directory name
project_dir = '/content/yolo2'

if not os.path.exists(project_dir):
    # Clone the correct repository
    !git clone https://github.com/victornaguiar/yolo2.git {project_dir}

# Change the current directory to your project directory
%cd {project_dir}

# Install project dependencies from your requirements.txt
!pip install -r requirements.txt

Cloning into '/content/yolo2'...
remote: Enumerating objects: 56, done.
remote: Counting objects: 100% (56/56), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 56 (delta 14), reused 41 (delta 5), pack-reused 0 (from 0)
Receiving objects: 100% (56/56), 58.90 KiB | 1.34 MiB/s, done.
Resolving deltas: 100% (14/14), done.
/content/yolo2
  Cloning https://github.com/JonathonLuiten/TrackEval.git to /tmp/pip-req-build-j4dtvslj
  Running command git clone --filter=blob:none --quiet https://github.com/JonathonLuiten/TrackEval.git /tmp/pip-req-build-j4dtvslj
  Resolved https://github.com/JonathonLuiten/TrackEval.git to commit 12c8791b303e0a0b50f753af204249e622d0281a
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 31.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━

## 4. Verify Installation and Hardware

In [7]:
# Check hardware and installations
import torch
import cv2
import numpy as np
from ultralytics import YOLO

print("=== Hardware Information ===")
print(f"Python version: {torch.__version__}")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU count: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        gpu_name = torch.cuda.get_device_name(i)
        gpu_memory = torch.cuda.get_device_properties(i).total_memory / (1024**3)
        print(f"  GPU {i}: {gpu_name} ({gpu_memory:.1f} GB)")

print(f"\n=== Library Versions ===")
print(f"OpenCV version: {cv2.__version__}")
print(f"NumPy version: {np.__version__}")

# Test YOLO
try:
    model = YOLO('yolov8n.pt')
    print(f"YOLO model loaded successfully")
except Exception as e:
    print(f"Error loading YOLO: {e}")

# Test BotSort
try:
    from boxmot import BotSORT
    print(f"BotSORT available")
except ImportError:
    print(f"BotSORT not available - install with: pip install boxmot")

print("\n✓ All core libraries loaded successfully!")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
=== Hardware Information ===
Python version: 2.6.0+cu124
PyTorch version: 2.6.0+cu124
CUDA available: True
CUDA version: 12.4
GPU count: 1
  GPU 0: NVIDIA A100-SXM4-40GB (39.6 GB)

=== Library Versions ===
OpenCV version: 4.12.0
NumPy version: 2.0.2


100%|██████████| 6.25M/6.25M [00:00<00:00, 78.6MB/s]


YOLO model loaded successfully
BotSORT not available - install with: pip install boxmot

✓ All core libraries loaded successfully!


## 5. Download Sample Data and Models

In [8]:
# Download sample data and models
!python scripts/download_models.py --all

python3: can't open file '/content/scripts/download_models.py': [Errno 2] No such file or directory


## 6. Set Up SSH for Remote Development (Optional)

In [9]:
# Install and configure SSH for VS Code remote development
!pip install colab-ssh -q

print("SSH setup ready. To connect VS Code remotely, run:")
print("")
print("from colab_ssh import launch_ssh_cloudflared")
print("launch_ssh_cloudflared(password='your_secure_password')")
print("")
print("Then follow the instructions to connect VS Code.")

SSH setup ready. To connect VS Code remotely, run:

from colab_ssh import launch_ssh_cloudflared
launch_ssh_cloudflared(password='your_secure_password')

Then follow the instructions to connect VS Code.


In [10]:
# Uncomment and run this cell to start SSH server
# from colab_ssh import launch_ssh_cloudflared
# launch_ssh_cloudflared(password="your_secure_password_here")

## 7. Copy Dataset from Google Drive to Local Storage

In [11]:
# Copy dataset from Google Drive to local SSD for faster access
import shutil
from pathlib import Path

gdrive_dataset = "/content/drive/MyDrive/SOCCER_DATA/deepsort_dataset_train"
local_dataset = "/content/soccer_dataset"

if os.path.exists(gdrive_dataset):
    print(f"Copying dataset from Google Drive to local SSD...")
    print(f"Source: {gdrive_dataset}")
    print(f"Destination: {local_dataset}")

    if os.path.exists(local_dataset):
        print("Local dataset already exists. Skipping copy.")
    else:
        shutil.copytree(gdrive_dataset, local_dataset)
        print("Dataset copied successfully!")

    # Verify dataset structure
    print("\nDataset structure:")
    for root, dirs, files in os.walk(local_dataset):
        level = root.replace(local_dataset, '').count(os.sep)
        indent = ' ' * 2 * level
        print(f"{indent}{os.path.basename(root)}/")
        subindent = ' ' * 2 * (level + 1)
        for file in files[:5]:  # Show first 5 files only
            print(f"{subindent}{file}")
        if len(files) > 5:
            print(f"{subindent}... and {len(files) - 5} more files")
else:
    print(f"Dataset not found at: {gdrive_dataset}")
    print("Please upload your dataset to Google Drive first.")

Copying dataset from Google Drive to local SSD...
Source: /content/drive/MyDrive/SOCCER_DATA/deepsort_dataset_train
Destination: /content/soccer_dataset
Dataset copied successfully!

Dataset structure:
soccer_dataset/
  tracking_results/
    SNMOT-098.txt
    SNMOT-153.txt
    SNMOT-065.txt
    SNMOT-160.txt
    SNMOT-070.txt
    ... and 52 more files
  sequences/
    SNMOT-164/
      000446.jpg
      000559.jpg
      000153.jpg
      000420.jpg
      000342.jpg
      ... and 746 more files
    SNMOT-099/
      000446.jpg
      000559.jpg
      000153.jpg
      000420.jpg
      000342.jpg
      ... and 746 more files
    SNMOT-107/
      000446.jpg
      000559.jpg
      000153.jpg
      000420.jpg
      000342.jpg
      ... and 746 more files
    SNMOT-075/
      000446.jpg
      000559.jpg
      000153.jpg
      000420.jpg
      000342.jpg
      ... and 746 more files
    SNMOT-158/
      000446.jpg
      000559.jpg
      000153.jpg
      000420.jpg
      000342.jpg
      ... and 746

## 8. Test the Pipeline

In [20]:
# --- Definitive Fix for ModuleNotFoundError & FileNotFoundError ---
import os
import sys

# 1. Define the project directory
project_dir = '/content/yolo2'

# 2. Change the current working directory (good practice)
%cd {project_dir}

# 3. Add the project directory to Python's path (CRITICAL STEP)
if project_dir not in sys.path:
    sys.path.insert(0, project_dir)

print(f"Current working directory: {os.getcwd()}")
print(f"Python is now looking for modules in: {sys.path[0]}")

# 4. Install the missing dependency
print("\nInstalling boxmot...")
!pip install -q boxmot

# 5. Download the sample video (FIX for FileNotFoundError)
print("\nDownloading sample video...")
sample_video_path = 'data/sample_videos/people.mp4'
if not os.path.exists(sample_video_path):
    os.makedirs(os.path.dirname(sample_video_path), exist_ok=True)
    # Download a sample video of people walking
    !wget -q -O {sample_video_path} "https://videos.pexels.com/video-files/853881/853881-sd_640_360_30fps.mp4"
    print(f"Video downloaded to '{sample_video_path}'")
else:
    print("Sample video already exists.")


# --- Original code from the cell below (with fixes) ---
print("\nRunning a quick test of the tracking pipeline...")
from src.tracking import YOLOTracker
import cv2

if os.path.exists(sample_video_path):
    # Initialize tracker and video stream
    tracker = YOLOTracker(model_name='yolov8n.pt')
    cap = cv2.VideoCapture(sample_video_path)
    frame_count = 0

    print("Processing first 100 frames of sample video...")
    while cap.isOpened() and frame_count < 100:
        ret, frame = cap.read()
        if not ret:
            break

        # The tracker expects a list of detections, but for a quick test,
        # we can pass None and it will perform detection internally.
        tracks = tracker.update(None, frame)
        print(f"Frame {frame_count}: {len(tracks)} tracks detected")
        frame_count += 1

    cap.release()
    print("\n✓ Pipeline test completed successfully!")

else:
    print(f"❌ ERROR: Sample video not found at: {sample_video_path}")
    print("Please make sure you have run the setup cells correctly.")

/content/yolo2
Current working directory: /content/yolo2
Python is now looking for modules in: /content/yolo2

Installing boxmot...

Sample video already exists.

Running a quick test of the tracking pipeline...


100%|██████████| 6.25M/6.25M [00:00<00:00, 76.4MB/s]


YOLO tracker initialized with yolov8n.pt on cuda
Processing first 100 frames of sample video...

✓ Pipeline test completed successfully!


In [24]:
print("it's over anakin, i have the high ground \nyou underestimate my power \ndon't try it")

it's over anakin, i have the high ground 
you underestimate my power 
don't try it


## Next Steps

Your environment is now set up! You can:

1. **Run the simple tracking demo**: Open `02_simple_tracking_demo.ipynb`
2. **Process soccer datasets**: Open `03_soccer_tracking_pipeline.ipynb`
3. **Evaluate results**: Open `04_evaluation_analysis.ipynb`
4. **Use command-line scripts**:
   - Track videos: `python scripts/run_tracking.py --help`
   - Evaluate results: `python scripts/evaluate_results.py --help`

### For Remote Development:
- Connect VS Code using the SSH tunnel created above
- Access files on the VM's SSD for fast processing
- Leverage GPU acceleration automatically

### Performance Tips:
- Use the local SSD (`/content/`) for active datasets
- Keep original data on Google Drive for backup
- Monitor GPU memory usage with `nvidia-smi`